In [ ]:
# Set up matplotlib for jupyter notebook
%matplotlib inline

In [ ]:
# Set up dependencies
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
import tweepy
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [ ]:
# Set up Tweepy Auth with JSON parser
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
# Set up variables necessary for 5-min check, target term history, filtering tweets that are not in english;
x = 0
tweet_id_1 = None
tweet_id_2 = None
target_term = ""
requesting_user = ""
target_term_history = []
tweet_list = []
oldest_tweet = None
b = 1
lang = "en"
# For every 5 minutes, get the most recent one
while(b):
    while(True):
        if x == 1:
            x = 0
            try:
                # Scan for mention to analyze and use since_id to get the latest mention
                response = api.mentions_timeline(since_id=tweet_id_1, count=1)
                # Update the value with every new mention grabbed
                tweet_id_1 = response[0]["id"]
                # Grab target term
                target_term = response[0]["text"].split("@")[2]
                requesting_user = response[0]["user"]["screen_name"]
                # Make it so that if a term has already been searched once,
                # it won't get searched again
                if target_term not in target_term_history:
                    target_term_history.append(target_term)
                    # Grab the tweets, 100 for each of 5 pages
                    for x in range(5):
                        public_tweets = api.user_timeline(target_term, 
                                                          count=100, 
                                                          result_type="recent",
                                                          max_id=oldest_tweet)
                        # For each item within my first page of tweets...
                        for i in range(len(public_tweets)):
                            # Filter tweets that are not in English (due to limitations on VADER)
                            if public_tweets[i]["lang"] == "en":
                                # Append the text to my list
                                tweet_list.append(public_tweets[i]["text"])
                                # Store the ID of the latest tweet
                                tweet_id_2 = public_tweets[i]["id"]
                                # Subtract the ID by 1 to get the max_id for the next page
                                oldest_tweet = tweet_id_2 - 1
                        # Don't want to overload servers
                        time.sleep(1)
                # Perform sentimental analysis on the tweets
                compound_list = []
                for text in tweet_list:
                    results = analyzer.polarity_scores(text)
                    compound_list.append(results["compound"])
                # Plot the analysis
                numb_of_tweets_X = len(compound_list)
                current_date = datetime.today().strftime("%m/%d/%Y")
                plt.plot(range(numb_of_tweets_X), compound_list, c="#A61515", marker="o", linewidth=.5)
                plt.grid(alpha=1)
                plt.title(f"Sentimental Analysis of Tweets ({current_date})")
                plt.xlabel("Tweets Ago")
                plt.ylabel("Tweet Polarity")
                plt.legend([f"@{target_term}"], loc="upper center", title="Tweets", bbox_to_anchor=(1.28, 1))
                plt.savefig(f"sentimental_analysis_{target_term}.png", dpi=300, bbox_inches="tight")
                # Push the .png to Twitter and post the analysis
                api.update_with_media(f"sentimental_analysis_{target_term}.png",
                                      f"Here is your analysis of @{target_term}, {requesting_user}!")
                time.sleep(300)
                x = 1
            # If the target term is in the target history, then this exception will fire
            except:
                print("Empty list: Most recent tweet is already analyzed.")
                print("Five minutes until next check for most recent tweet.")
                b = 0
                time.sleep(300)
                b = 1
        else: x += 1